In [ ]:
from tensorflow.keras.optimizers import Adam,RMSprop
from tensorflow.keras import models, Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.metrics import Balanceacc

from models import efficientNetV2B0_model, efficientNetV2B3_model, vgg16_model, resnet50_model, mobilenetv2_model, frank_model
from config import efficientNet_config

import cv2 as cv
import numpy as np
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder

import os
# 使用第一張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
height  = efficientNet_config['height_B0']
width   = efficientNet_config['width_B0']
input_shape  = efficientNet_config['input_shape_B0']

# 讀取資料

In [ ]:
# label
tra_df = pd.read_csv("../dataset/training_dataset/direc/train/new_class.csv", index_col=False)
val_df = pd.read_csv("../dataset/training_dataset/direc/validation/new_class.csv", index_col=False)
# 不要 label='right'的資料
tra_df = tra_df[tra_df['new_class']!='right']
val_df = val_df[val_df['new_class']!='right']

tra_img_labels = tra_df['new_class'].values.tolist()
val_img_labels = val_df['new_class'].values.tolist()

# label one hot
labels_onehot  = OneHotEncoder().fit_transform(np.array(tra_img_labels+val_img_labels).reshape(-1,1)).toarray()

r,c = tra_df.shape
tra_img_labels_onehot = labels_onehot[:r]
val_img_labels_onehot = labels_onehot[r:]
print('訓練集label 維度= ',tra_img_labels_onehot.shape)
print('驗證集label 維度= ',val_img_labels_onehot.shape)

In [ ]:
tra_img_name_list = tra_df['img_name'].values.tolist()
val_img_name_list = val_df['img_name'].values.tolist() 

In [ ]:
# image dirction
tra_dir = '../dataset/training_dataset/img/train/CropSmooth/'
val_dir = '../dataset/training_dataset/img/validation/CropSmooth/'

tra_img_arrays = []
val_img_arrays = []

## train image array
for img_name in tra_img_name_list:
    # image
    img_array = cv.imread(tra_dir+img_name)
    img_array = cv.resize(img_array,(height,width))      # resize to (224,224)
    tra_img_arrays.append(img_array)

    # npy
    # img_array = np.load(tra_dir + img_name.split(".")[0] + ".npy")
    # img_array = cv.resize(img_array,(height,width))
    # tra_img_arrays.append(img_array)

## validation
for img_name in val_img_name_list:
    # imege
    img_array = cv.imread(val_dir+img_name)
    img_array = cv.resize(img_array,(height,width))      # resize to (224,224)
    val_img_arrays.append(img_array)

    # npy
    # img_array = np.load(val_dir + img_name.split(".")[0] + ".npy")
    # img_array = cv.resize(img_array,(height,width))
    # val_img_arrays.append(img_array)


tra_img_arrays = np.array(tra_img_arrays) 
val_img_arrays = np.array(val_img_arrays)

# npy
# tra_img_arrays = np.repeat(tra_img_arrays[..., np.newaxis], 3, -1)   # 灰階轉RGB
# val_img_arrays = np.repeat(val_img_arrays[..., np.newaxis], 3, -1)   # 灰階轉RGB

In [ ]:
print('訓練集數量= ',len(tra_img_arrays))
print('驗證集數量= ',len(val_img_arrays))


In [ ]:
# tra_img_arrays, tra_img_labels = shuffle(tra_img_arrays, tra_img_labels)
# val_img_arrays, val_img_labels = shuffle(val_img_arrays, val_img_labels)

In [ ]:
print('訓練集維度= ', tra_img_arrays.shape)
print('驗證集維度= ', val_img_arrays.shape)
print('訓練集label維度= ', tra_img_labels_onehot.shape)
print('驗證集label維度= ', val_img_labels_onehot.shape)

In [ ]:
from utils import angle_error

In [ ]:
model = efficientNetV2B0_model(cls=tra_img_labels_onehot.shape[1])

model.compile(loss='categorical_crossentropy', 
              optimizer= Adam(learning_rate=1e-5),
              metrics=['accuracy', 
                        # Balanceacc(),
                        angle_error
                      ]
              )


In [ ]:
day = '20221130'
checkpoint_filepath = '../model/202211/{}.weights'.format(day)
model_checkpoint_callback = ModelCheckpoint(
        filepath=checkpoint_filepath,
        save_weights_only=True,
        monitor='val_angle_error',
        mode='min',
        save_best_only=True)

# learning rate 降低
# reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
#                               factor=0.1,
#                               patience=5, 
#                               min_lr=1e-8)



callbacks = [model_checkpoint_callback, 
        #      reduce_lr
             ]

In [ ]:
# 訓練
class_weight = {0: 1.1,
                1: 1,
                2: 1.4
                }
epochs = 100
history = model.fit(
      x = tra_img_arrays.astype('float'),
      y = tra_img_labels_onehot,
      validation_data = (val_img_arrays.astype('float'), val_img_labels_onehot),      
      epochs          = epochs,
      verbose         = 1,
      callbacks       = callbacks,
      # class_weight    = class_weight
)

# 畫圖

In [ ]:
loss         = history.history['loss']
val_loss     = history.history['val_loss']
accuracy     = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
BlA          = history.history['angle_error']        # 先畫圖
val_BlA      = history.history['val_angle_error']    # 先畫圖

In [ ]:
x = [i for i in range(1, epochs+1)]

min_valloss_x = val_loss.index(min(val_loss)) + 1
min_valloss_y = min(val_loss)

plt.figure(figsize=(24,4))
plt.plot(x, loss, 'r')     
plt.plot(x, val_loss, 'b')     # red dotted line (no marker)

plt.plot(min_valloss_x, min_valloss_y, 'd', color='g')
plt.text(min_valloss_x, min_valloss_y, "({},{})".format(min_valloss_x,round(min_valloss_y,2)), ha='left',va='top',fontsize=20)

plt.legend(['loss','val_loss'])
# plt.ylim((0, 0.01))
plt.xlabel('epoch')
plt.ylabel('loss')
plt.title('loss')
plt.savefig('../model/202211/{}_loss.png'.format(day),
            bbox_inches='tight',
            pad_inches=1,
            transparent=False)
plt.show()

In [ ]:
x = [i for i in range(1, epochs+1)]

max_valacc_x = val_accuracy.index(max(val_accuracy)) + 1
max_valacc_y = max(val_accuracy)

plt.figure(figsize=(24,4))
plt.plot(x, accuracy, 'r')     
plt.plot(x, val_accuracy, 'b')     # red dotted line (no marker)

plt.plot(max_valacc_x, max_valacc_y, 'd', color='g')
plt.text(max_valacc_x, max_valacc_y, "({},{})".format(max_valacc_x, round(max_valacc_y,2)), ha='left',va='top',fontsize=20)

plt.legend(['accuracy','val_accuracy'])
plt.ylim((0, 1.1))
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.title('accuracy')
plt.savefig('../model/202211/{}_acc.png'.format(day),
            bbox_inches='tight',
            pad_inches=1,
            transparent=False)
plt.show()

In [ ]:
x = [i for i in range(1, epochs+1)]

max_valBlA_x = val_BlA.index(min(val_BlA)) + 1
max_valBlA_y = min(val_BlA)

plt.figure(figsize=(24,4))
plt.plot(x, BlA, 'r')     
plt.plot(x, val_BlA, 'b')     # red dotted line (no marker)

plt.plot(max_valBlA_x, max_valBlA_y, 'd', color='g')
plt.text(max_valBlA_x, max_valBlA_y, "({},{})".format(max_valBlA_x, round(max_valBlA_y,2)), ha='left',va='top',fontsize=20)

plt.legend(['angle error','val_angle error'])
plt.ylim((0, 1.1))
plt.xlabel('epoch')
plt.ylabel('angle error')
plt.title('angle error')
plt.savefig('../model/202211/{}_BlA.png'.format(day),
            bbox_inches='tight',
            pad_inches=1,
            transparent=False)
plt.show()

In [ ]:
import json
# Get the dictionary containing each metric and the loss for each epoch
history_dict = history.history
# Save it under the form of a json file
json.dump(history_dict, open('../model/202211/{}.history'.format(day), 'w'))